In [ ]:
pip install simcse transformers==4.2.1 scipy==1.5.4 datasets==1.2.1 pandas==1.1.5 scikit-learn==0.24.0 prettytable==2.1.0 gradio torch setuptools==49.3.0

     |████████████████████████████████| 1.8 MB 5.1 MB/s 
     |████████████████████████████████| 25.9 MB 1.4 MB/s 
     |████████████████████████████████| 159 kB 58.6 MB/s 
     |████████████████████████████████| 22.3 MB 1.3 MB/s 
     |████████████████████████████████| 979 kB 46.5 MB/s 
     |████████████████████████████████| 790 kB 59.3 MB/s 
     |████████████████████████████████| 2.9 MB 37.8 MB/s 
     |████████████████████████████████| 895 kB 40.8 MB/s 
     |████████████████████████████████| 69 kB 7.0 MB/s 
     |████████████████████████████████| 243 kB 57.8 MB/s 
     |████████████████████████████████| 1.9 MB 45.5 MB/s 
     |████████████████████████████████| 206 kB 53.5 MB/s 
     |████████████████████████████████| 961 kB 35.6 MB/s 
     |████████████████████████████████| 63 kB 2.6 MB/s 
     |████████████████████████████████| 3.6 MB 45.4 MB/s 
  Created wheel for simcse: filename=simcse-0.4-py3-none-any.whl size=15032 sha256=956ed4d068d630aa122b638fc6f18439533f6dd8a733b9d904aa

In [ ]:
import torch
import pandas as pd
import numpy as np
from simcse import SimCSE
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity, f1_score, matthews_corrcoef

from transformers import BertConfig, BertModel
from transformers import AutoModelForSequenceClassification, DistilBertForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from datasets import load_metric
import csv

if torch.cuda.is_available():
  device = "cuda:0"
else:
  device = "cpu:0"
dtype = torch.cuda.FloatTensor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_folder = 'drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/'

## Evaluation of triplets on dev set


## Eval - Bert base uncased - LR 5e-5 - 2 epoch

In [ ]:
model = SimCSE("/content/drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/twiBot-360k-similarity-metadata-2epoch_LR5e-5_bert_base_uncased")
# "drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/twiBot-360k-similarity-metadata-1epoch"

df_dev = pd.read_csv(data_folder + "/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/dev_all_data_triplets_with_similarity_corrected_metadata.csv" , usecols = ['sent0', 'sent1', 'hard_neg'])
n = len(df_dev)
# df_dev = df_dev[:n]
batch_size = 120000
count = 0
for i in range(0, len(df_dev), batch_size):
  print("{}/{}".format(i+1, len(df_dev)))
  encoded_sent0 = model.encode(list(df_dev[i:i+batch_size].sent0), device).cpu().numpy()
  encoded_sent1 = model.encode(list(df_dev[i:i+batch_size].sent1), device).cpu().numpy()
  encoded_hard_neg = model.encode(list(df_dev[i:i+batch_size].hard_neg), device).cpu().numpy()
  for j in range(len(encoded_sent0)):
    cos_sim_positive = cosine_similarity([encoded_sent0[j]], [encoded_sent1[j]])
    cos_sim_negative = cosine_similarity([encoded_sent0[j]], [encoded_hard_neg[j]])
    count += (cos_sim_positive > cos_sim_negative)[0][0]
  # print("Accuracy as of batch", str(i+1), str(count/i+batch_size))
print("Accuracy", count/n)

Some weights of BertModel were not initialized from the model checkpoint at /content/drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/twiBot-360k-similarity-metadata-2epoch_LR5e-5_bert_base_uncased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1/480000


  0%|          | 0/1875 [00:00<?, ?it/s]

120001/480000


  0%|          | 0/1875 [00:00<?, ?it/s]

240001/480000


  0%|          | 0/1875 [00:00<?, ?it/s]

360001/480000


100%|██████████| 1875/1875 [07:11<00:00,  4.34it/s]


Accuracy 0.9268416666666667


## Eval princeton-nlp/sup-simcse-bert-base-uncased - LR 5e-5 - 1 epoch

In [ ]:
model = SimCSE("/content/drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/twiBot-360k-similarity-metadata-1epoch_LR5e-5_sup-simcse-bert-base-uncased")
# "drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/twiBot-360k-similarity-metadata-1epoch"

df_dev = pd.read_csv(data_folder + "/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/dev_all_data_triplets_with_similarity_corrected_metadata.csv" , usecols = ['sent0', 'sent1', 'hard_neg'])
n = len(df_dev)
# df_dev = df_dev[:n]
batch_size = 120000
count = 0
for i in range(0, len(df_dev), batch_size):
  print("{}/{}".format(i+1, len(df_dev)))
  encoded_sent0 = model.encode(list(df_dev[i:i+batch_size].sent0), device).cpu().numpy()
  encoded_sent1 = model.encode(list(df_dev[i:i+batch_size].sent1), device).cpu().numpy()
  encoded_hard_neg = model.encode(list(df_dev[i:i+batch_size].hard_neg), device).cpu().numpy()
  for j in range(len(encoded_sent0)):
    cos_sim_positive = cosine_similarity([encoded_sent0[j]], [encoded_sent1[j]])
    cos_sim_negative = cosine_similarity([encoded_sent0[j]], [encoded_hard_neg[j]])
    count += (cos_sim_positive > cos_sim_negative)[0][0]
  # print("Accuracy as of batch", str(i+1), str(count/i+batch_size))
print("Accuracy", count/n)

Some weights of BertModel were not initialized from the model checkpoint at /content/drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/twiBot-360k-similarity-metadata-1epoch_LR5e-5_sup-simcse-bert-base-uncased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1/480000


  0%|          | 0/1875 [00:00<?, ?it/s]

120001/480000


  0%|          | 0/1875 [00:00<?, ?it/s]

240001/480000


  0%|          | 0/1875 [00:00<?, ?it/s]

360001/480000


100%|██████████| 1875/1875 [07:15<00:00,  4.31it/s]


Accuracy 0.9154666666666667


## Exploring

In [ ]:
model = SimCSE("princeton-nlp/sup-simcse-bert-base-uncased")
## is supervised and case insensitive

## Fine-tuning with Classifier

In [ ]:
!ls /content/drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/twiBot-360k-similarity-metadata-2epoch_LR5e-5_bert_base_uncased

config.json	   scheduler.pt		    training_args.bin
eval_results.txt   special_tokens_map.json  train_results.txt
optimizer.pt	   tokenizer_config.json    vocab.txt
pytorch_model.bin  trainer_state.json


In [ ]:
# Task 00 - contrastive learning embeddings without tuning classifier
model_twitter_classifier = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/twiBot-360k-similarity-metadata-1epoch_LR5e-5_sup-simcse-bert-base-uncased")
# prevent backprop for base model
for param in model_twitter_classifier.base_model.parameters():
    param.requires_grad = False

Some weights of the model checkpoint at /content/drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/twiBot-360k-similarity-metadata-1epoch_LR5e-5_sup-simcse-bert-base-uncased were not used when initializing BertForSequenceClassification: ['mlp.dense.weight', 'mlp.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Mo

In [ ]:
# Task 01 - 360k model, no freeze on params Bert-base-uncased-2epoch-LR5e-5" 
model_twitter_classifier = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/twiBot-360k-similarity-metadata-2epoch_LR5e-5_bert_base_uncased")
# prevent backprop for base model
for param in model_twitter_classifier.base_model.parameters():
    param.requires_grad = True

Some weights of the model checkpoint at /content/drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/twiBot-360k-similarity-metadata-2epoch_LR5e-5_bert_base_uncased were not used when initializing BertForSequenceClassification: ['mlp.dense.weight', 'mlp.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_

In [ ]:
training_args = TrainingArguments("test_trainer", evaluation_strategy="epoch")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/twiBot-360k-similarity-metadata-1epoch_LR5e-5_sup-simcse-bert-base-uncased")

In [ ]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
    

In [ ]:

df_train = pd.read_csv(data_folder + "Model_TwiBot_Similarity_With_Metadata_360k_Tweets/train_all_data_triplets_with_similarity_new.csv", usecols = ['sent0', 'sent0_label'])
df_train = df_train.drop_duplicates()
df_train = df_train.sample(frac=1).reset_index(drop=True)

df_dev = pd.read_csv(data_folder + "/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/dev_all_data_triplets_with_similarity_corrected_metadata.csv" , usecols = ['sent0', 'sent0_label'])
df_dev = df_dev.drop_duplicates()
df_dev = df_dev.sample(frac=1).reset_index(drop=True)


In [ ]:
train_sentences = list(df_train.sent0) #["rt <_user_> : hahaha <_user_> i got my revenge do n't worry 😂 😂 <_user_> <_url_>", "<_user_> <_user_> i was involved in the crash mentionned in the news", "election results : see the final results and what votes are still being counted in the races that matter to you across the ohio valley . <_url_>"]
train_labels = df_train.sent0_label #[0, 0, 1]

dev_sentences = list(df_dev.sent0)
dev_labels = df_dev.sent0_label

In [ ]:
class TwiDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_tokens = tokenizer(train_sentences, padding=True, truncation=True)
train_dataset = TwiDataset(train_tokens, train_labels)

# dev_tokens = tokenizer(dev_sentences, padding=True, truncation=True)
# dev_dataset = TwiDataset(dev_tokens, dev_labels)


# test_dataset = TwiDataset(test_encodings, test_labels)

## Train classifier task 00 - bert-base-uncased-1epoch-LR5e-5

In [ ]:
training_args = TrainingArguments(
    output_dir = data_folder+'Model_TwiBot_Similarity_With_Metadata_360k_Tweets/Classifier/ \
                            twiBot-360k-similarity-metadata-1epoch_LR5e-5_bert_base_uncased/results',          # output directory
    num_train_epochs = 1,              # total number of training epochs
    per_device_train_batch_size = 16,  # batch size per device during training
    per_device_eval_batch_size = 64,   # batch size for evaluation
    warmup_steps = 500,                # number of warmup steps for learning rate scheduler
    weight_decay = 0.01,               # strength of weight decay
    logging_dir = data_folder+'Model_TwiBot_Similarity_With_Metadata_360k_Tweets/Classifier/ \
                            twiBot-360k-similarity-metadata-1epoch_LR5e-5_bert_base_uncased/logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model_twitter_classifier,       # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset           # training dataset
    # eval_dataset=val_dataset            # evaluation dataset
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/_distutils_hack/__init__.py:19: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  "Distutils was imported before Setuptools. This usage is discouraged "


Step,Training Loss
10,0.668900
20,0.667800
30,0.658100
40,0.679400
50,0.660300
60,0.658200
70,0.642700
80,0.663500
90,0.662000
100,0.658900


TrainOutput(global_step=22145, training_loss=0.03263004273605161, metrics={'train_runtime': 4978.6797, 'train_samples_per_second': 4.448, 'total_flos': 88910428149525264, 'epoch': 1.0})

## Train classifier task 00 - bert-base-uncased-2epoch-LR5e-5

In [ ]:
training_args = TrainingArguments(
    output_dir = data_folder+'Model_TwiBot_Similarity_With_Metadata_360k_Tweets/Classifier/ \
                            twiBot-360k-similarity-metadata-2epoch_LR5e-5_bert_base_uncased/results',          # output directory
    num_train_epochs = 2,              # total number of training epochs
    per_device_train_batch_size = 16,  # batch size per device during training
    per_device_eval_batch_size = 64,   # batch size for evaluation
    warmup_steps = 500,                # number of warmup steps for learning rate scheduler
    weight_decay = 0.01,               # strength of weight decay
    logging_dir = data_folder+'Model_TwiBot_Similarity_With_Metadata_360k_Tweets/Classifier/ \
                            twiBot-360k-similarity-metadata-2epoch_LR5e-5_bert_base_uncased/logs',            # directory for storing logs
    logging_steps=200,
)

trainer = Trainer(
    model=model_twitter_classifier,       # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset           # training dataset
    # eval_dataset=val_dataset            # evaluation dataset
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/_distutils_hack/__init__.py:19: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  "Distutils was imported before Setuptools. This usage is discouraged "


Step,Training Loss
200,0.673100
400,0.583400
600,0.440600
800,0.305000
1000,0.222700
1200,0.171400
1400,0.137900
1600,0.113100
1800,0.094100
2000,0.078400


TrainOutput(global_step=44290, training_loss=0.020996497264966708, metrics={'train_runtime': 9930.1626, 'train_samples_per_second': 4.46, 'total_flos': 177820856299050528, 'epoch': 2.0})

## Train classifier task 00 - sup-simcse-bert-base-uncased-1epoch-LR5e-5

In [ ]:
training_args = TrainingArguments(
    output_dir = data_folder+'Model_TwiBot_Similarity_With_Metadata_360k_Tweets/Classifier/twiBot-360k-similarity-metadata-1epoch_LR5e-5_sup-simcse-bert-base-uncased/results',          # output directory
    num_train_epochs = 1,              # total number of training epochs
    per_device_train_batch_size = 16,  # batch size per device during training
    per_device_eval_batch_size = 64,   # batch size for evaluation
    warmup_steps = 500,                # number of warmup steps for learning rate scheduler
    weight_decay = 0.01,               # strength of weight decay
    logging_dir = data_folder+'Model_TwiBot_Similarity_With_Metadata_360k_Tweets/Classifier/twiBot-360k-similarity-metadata-1epoch_LR5e-5_sup-simcse-bert-base-uncased/logs',            # directory for storing logs
    logging_steps=200,
)

trainer = Trainer(
    model=model_twitter_classifier,       # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset           # training dataset
    # eval_dataset=val_dataset            # evaluation dataset
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/_distutils_hack/__init__.py:19: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  "Distutils was imported before Setuptools. This usage is discouraged "


Step,Training Loss
200,0.693400
400,0.667600
600,0.619400
800,0.566900
1000,0.533500
1200,0.500500
1400,0.472700
1600,0.453600
1800,0.438200
2000,0.420900


Step,Training Loss
200,0.693400
400,0.667600
600,0.619400
800,0.566900
1000,0.533500
1200,0.500500
1400,0.472700
1600,0.453600
1800,0.438200
2000,0.420900


TrainOutput(global_step=22145, training_loss=0.3414644513073127, metrics={'train_runtime': 4964.2614, 'train_samples_per_second': 4.461, 'total_flos': 88910428149525264, 'epoch': 1.0})

## Train classifier task 01 (360k model, no freeze on params) - bert-base-uncased-2epoch-LR5e-5

In [ ]:
import shutil
shutil.rmtree('/content/drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/Classifier/01-nofreeze-with-val-twiBot-360k-similarity-metadata-2epoch_LR5e-5_bert_base_uncased')

In [ ]:
training_args = TrainingArguments(
    output_dir = data_folder+'Model_TwiBot_Similarity_With_Metadata_360k_Tweets/Classifier/01-nofreeze-with-val-twiBot-360k-similarity-metadata-2epoch_LR5e-5_bert_base_uncased/results',          # output directory
    num_train_epochs = 1,              # total number of training epochs
    per_device_train_batch_size = 16,  # batch size per device during training
    per_device_eval_batch_size = 64,   # batch size for evaluation
    warmup_steps = 500,                # number of warmup steps for learning rate scheduler
    weight_decay = 0.01,               # strength of weight decay
    logging_dir = data_folder+'Model_TwiBot_Similarity_With_Metadata_360k_Tweets/Classifier/01-nofreeze-with-val-twiBot-360k-similarity-metadata-2epoch_LR5e-5_bert_base_uncased/logs',            # directory for storing logs
    logging_steps=5000,
    evaluation_strategy="steps"
)

trainer = Trainer(
    model=model_twitter_classifier,       # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=dev_dataset,            # evaluation dataset
    compute_metrics=compute_metrics
)

trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
5000,0.022400,1.181293,0.819554,1546.869000,76.073000
10000,0.011000,1.474235,0.810325,1546.970700,76.068000
15000,0.006700,1.434572,0.852288,1546.906700,76.071000


12/01/2021 13:59:52 - INFO - /usr/local/lib/python3.7/dist-packages/datasets/metric.py -   Removing /root/.cache/huggingface/metrics/accuracy/default/default_experiment-1-0.arrow
12/01/2021 15:20:25 - INFO - /usr/local/lib/python3.7/dist-packages/datasets/metric.py -   Removing /root/.cache/huggingface/metrics/accuracy/default/default_experiment-1-0.arrow
12/01/2021 16:40:58 - INFO - /usr/local/lib/python3.7/dist-packages/datasets/metric.py -   Removing /root/.cache/huggingface/metrics/accuracy/default/default_experiment-1-0.arrow


KeyboardInterrupt: ignored

## Copy latest checkpoint folder

In [ ]:
!cp -R /content/drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/Classifier/twiBot-360k-similarity-metadata-1epoch_LR5e-5_sup-simcse-bert-base-uncased/results/checkpoint-22000 /content/drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/Classifier/twiBot-360k-similarity-metadata-1epoch_LR5e-5_sup-simcse-bert-base-uncased/classifier_model

## Prediction

In [ ]:
# twiBot-360k-similarity-metadata-1epoch_LR5e-5_sup-simcse-bert-base-uncased
path_classifier = "/content/drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/Classifier/twiBot-360k-similarity-metadata-1epoch_LR5e-5_sup-simcse-bert-base-uncased/classifier_model/checkpoint-22000"
path_tokenizer = 'drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/twiBot-360k-similarity-metadata-1epoch_LR5e-5_sup-simcse-bert-base-uncased'

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(path_classifier)
training_args = TrainingArguments("test_trainer", evaluation_strategy="epoch")
tokenizer = AutoTokenizer.from_pretrained(path_tokenizer)

In [ ]:
df_dev = pd.read_csv(data_folder + "/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/dev_all_data_triplets_with_similarity_corrected_metadata.csv" , usecols = ['sent0', 'sent0_label'])

In [ ]:
pd.set_option("display.max_rows", None)
pd.set_option('display.max_colwidth', None)

In [ ]:
print(len(df_dev))
df_dev = df_dev.drop_duplicates()
print(len(df_dev))
print(df_dev.iloc[0].to_string)

480000
117675
<bound method Series.to_string of sent0          RAkufoAddo unprotected unverified defprof nondefimg 388018 109 37 221 1131 My fervent wish today is for us to eschew division, and unite as a people, as we strive collectively for progress and prosperity. May patriotism and compassion forever be our guiding principles. Happy Independence Day! <url>
sent0_label                                                                                                                                                                                                                                                                                                              0
Name: 0, dtype: object>


In [ ]:
pred_csv_file="/content/drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/Classifier/twiBot-360k-similarity-metadata-2epoch_LR5e-5_bert_base_uncased/classifier_model/checkpoint-44000/pred_dev_twiBot-360k-similarity-metadata-2epoch_LR5e-5_bert_base_uncased.csv"

## Dev -  Prediction bert-base-uncased-2epoch-LR5e-5

In [ ]:
#Check if we need to freeze params
model.eval()
pred_curated = []
batch_size = 45 # 117675 / 45 = 1569
count = 0
for i in range(0, len(df_dev), batch_size):
  print("{}/{}".format(int((i/batch_size)+1), int(len(df_dev)/batch_size)))
  pt_batch = tokenizer(
      list(df_dev[i:i+batch_size].sent0), #["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
      padding=True,
      truncation=True,
      max_length=512,
      return_tensors="pt"
  )
  with torch.no_grad():
    prediction_logits = model(**pt_batch)
    prediction_results = torch.softmax(prediction_logits[0], dim=1)
    prediction_results = torch.argmax(prediction_results, dim=1)
    pred_curated.extend([[i,j] for i,j in zip(prediction_results.tolist(),df_dev[i:i+batch_size].sent0_label.tolist())])
    # print(prediction_results.numpy().shape, np.array(df_dev[i:i+batch_size].sent0_label).shape )
    count += np.sum(prediction_results.numpy() == np.array(df_dev[i:i+batch_size].sent0_label))
    print("Accuracy", count/(i+batch_size))
  if (int((i/batch_size)+1)%5 ==0):
    with open(pred_csv_file,"a") as f:
      # x=pickle.load(f)
      # x.extend(pred_cul)
      # pickle.dump(pred_cul, f)
      # pred_cul = []
      f_writer=csv.writer(f)
      f_writer.writerows(pred_curated)
      pred_curated =[]
    print("last batch saved", str(int((i/batch_size)+1)%100))
print("Final Accuracy=", count/len(df_dev))

Streaming output truncated to the last 5000 lines.
344/2615
Accuracy 0.8488372093023255
345/2615
Accuracy 0.8488888888888889
last batch saved 45
346/2615
Accuracy 0.8486191393705844
347/2615
Accuracy 0.8485430675632405
348/2615
Accuracy 0.8485951468710089
349/2615
Accuracy 0.8487106017191977
350/2615
Accuracy 0.8489523809523809
last batch saved 50
351/2615
Accuracy 0.8488762266540044
352/2615
Accuracy 0.8487373737373738
353/2615
Accuracy 0.8486622599937047
354/2615
Accuracy 0.8484620213433772
355/2615
Accuracy 0.8483255086071988
last batch saved 55
356/2615
Accuracy 0.8482521847690387
357/2615
Accuracy 0.8483037659508248
358/2615
Accuracy 0.8481067659838609
359/2615
Accuracy 0.8482822655524606
360/2615
Accuracy 0.8482716049382716
last batch saved 60
361/2615
Accuracy 0.84844567559249
362/2615
Accuracy 0.8484346224677717
363/2615
Accuracy 0.8481175390266299
364/2615
Accuracy 0.8481684981684982
365/2615
Accuracy 0.8482800608828006
last batch saved 65
366/2615
Accuracy 0.8482695810564663


In [ ]:
from sklearn.metrics import f1_score,matthews_corrcoef
pred_df = pd.read_csv(pred_csv_file, header=None)
if (len(pred_df) != len(df_dev)):
  print("MISMATCH")
print("f1_score",f1_score(pred_df[1].tolist(), pred_df[0].tolist()))
print("MCC", matthews_corrcoef(pred_df[1].tolist(), pred_df[0].tolist()))
# f1_score(pred_df[1].tolist(), pred_df[0].tolist(), average=None)

f1_score 0.8650303549480427
MCC 0.6993975611535929


## Dev -  Prediction 1epoch_LR5e-5_sup-simcse-bert-base-uncased

In [ ]:
#Check if we need to freeze params
model.eval()
pred_curated = []
batch_size = 45 # 117675 / 45 = 1569
count = 0
for i in range(0, len(df_dev), batch_size):
  print("{}/{}".format(int((i/batch_size)+1), int(len(df_dev)/batch_size)))
  pt_batch = tokenizer(
      list(df_dev[i:i+batch_size].sent0), #["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
      padding=True,
      truncation=True,
      max_length=512,
      return_tensors="pt"
  )
  with torch.no_grad():
    prediction_logits = model(**pt_batch)
    prediction_results = torch.softmax(prediction_logits[0], dim=1)
    prediction_results = torch.argmax(prediction_results, dim=1)
    pred_curated.extend([[i,j] for i,j in zip(prediction_results.tolist(),df_dev[i:i+batch_size].sent0_label.tolist())])
    # print(prediction_results.numpy().shape, np.array(df_dev[i:i+batch_size].sent0_label).shape )
    count += np.sum(prediction_results.numpy() == np.array(df_dev[i:i+batch_size].sent0_label))
    print("Accuracy", count/(i+batch_size))
  if (int((i/batch_size)+1)%5 ==0):
    with open(pred_csv_file,"a") as f:
      # x=pickle.load(f)
      # x.extend(pred_cul)
      # pickle.dump(pred_cul, f)
      # pred_cul = []
      f_writer=csv.writer(f)
      f_writer.writerows(pred_curated)
      pred_curated =[]
    print("last batch saved", str(int((i/batch_size)+1)%100))
print("Final Accuracy=", count/len(df_dev))

Streaming output truncated to the last 5000 lines.
last batch saved 40
341/2615
Accuracy 0.837862495927012
342/2615
Accuracy 0.837946718648473
343/2615
Accuracy 0.8380304502753483
344/2615
Accuracy 0.8377906976744186
345/2615
Accuracy 0.8378743961352657
last batch saved 45
346/2615
Accuracy 0.8375722543352602
347/2615
Accuracy 0.8376560999039385
348/2615
Accuracy 0.8377394636015326
349/2615
Accuracy 0.837886023559376
350/2615
Accuracy 0.8379682539682539
last batch saved 50
351/2615
Accuracy 0.8379233934789491
352/2615
Accuracy 0.8377525252525253
353/2615
Accuracy 0.8376455775889203
354/2615
Accuracy 0.8374764595103578
355/2615
Accuracy 0.8373708920187793
last batch saved 55
356/2615
Accuracy 0.8373283395755305
357/2615
Accuracy 0.837223778400249
358/2615
Accuracy 0.8373060211049038
359/2615
Accuracy 0.8372640049520272
360/2615
Accuracy 0.8372222222222222
last batch saved 60
361/2615
Accuracy 0.8374884579870729
362/2615
Accuracy 0.8373848987108655
363/2615
Accuracy 0.8372819100091827
36

In [ ]:
pred_df = pd.read_csv(pred_csv_file, header=None)
if (len(pred_df) != len(df_dev)):
  print("MISMATCH")
print("f1_score",f1_score(pred_df[1].tolist(), pred_df[0].tolist()))
print("MCC", matthews_corrcoef(pred_df[1].tolist(), pred_df[0].tolist()))
# f1_score(pred_df[1].tolist(), pred_df[0].tolist(), average=None)

MISMATCH
f1_score 0.8591738814155355
MCC 0.6868416814111098


## Test - Prediction bert-base-uncased-2epoch-LR5e-5

In [ ]:
import csv

df_test = pd.read_csv(data_folder + "/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/test_all_data_corrected_metadata.csv" , usecols = ['tweet', 'label'])
test_pred_csv_file="/content/drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/Classifier/twiBot-360k-similarity-metadata-2epoch_LR5e-5_bert_base_uncased/classifier_model/checkpoint-44000/pred_test_twiBot-360k-similarity-metadata-2epoch_LR5e-5_bert_base_uncased.csv"

In [ ]:
print(len(df_test))
print(df_test.iloc[0])
df_test = df_test.sample(frac=1, random_state=1).reset_index(drop=True)
print(df_test.iloc[0])

88035
tweet    SharonIsrael10 unprotected unverified defprof nondefimg 16596 16944 1 42684 49757 @ArtValley818_ CaliRed.
label                                                                                                            1
Name: 0, dtype: object
tweet    StarleeKine unprotected verified nondefprof nondefimg 50338 706 715 27692 18609 @GudGreaf @WeHaveSnacks I get the lack of enthusiasm. it is not an ideal situation. But it is the only way forward in this moment
label                                                                                                                                                                                                                    0
Name: 0, dtype: object


In [ ]:
#Check if we need to freeze params
model.eval()
pred_curated = []
batch_size = 15 # 88035 / 15 = 5869
count = 0
for i in range(0, len(df_test), batch_size):
  print("{}/{}".format(int((i/batch_size)+1), int(len(df_test)/batch_size)))
  pt_batch = tokenizer(
      list(df_test[i:i+batch_size].tweet), #["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
      padding=True,
      truncation=True,
      max_length=512,
      return_tensors="pt"
  )
  with torch.no_grad():
    prediction_logits = model(**pt_batch)
    prediction_results = torch.softmax(prediction_logits[0], dim=1)
    prediction_results = torch.argmax(prediction_results, dim=1)
    pred_curated.extend([[i,j] for i,j in zip(prediction_results.tolist(),df_test[i:i+batch_size].label.tolist())])
    # print(prediction_results.numpy().shape, np.array(df_test[i:i+batch_size].label).shape )
    # print("prediction_results", prediction_results)
    # print("actual_results", df_test[i:i+batch_size].label)
    count += np.sum(prediction_results.numpy() == np.array(df_test[i:i+batch_size].label))
    # print("count", count, "curr_total", (i+batch_size))
    print("Accuracy", count/(i+batch_size))
  # if (int((i/batch_size)+1)%5 ==0):
    
    # print("last batch saved", str(int((i/batch_size)+1)%100))
with open(test_pred_csv_file,"a") as f:
  # x=pickle.load(f)
  # x.extend(pred_cul)
  # pickle.dump(pred_cul, f)
  # pred_cul = []
  f_writer=csv.writer(f)
  f_writer.writerows(pred_curated)
  pred_curated =[]
print("Final Accuracy=", count/len(df_test))

Streaming output truncated to the last 5000 lines.
Accuracy 0.86
3371/5869
Accuracy 0.859982201127262
3372/5869
Accuracy 0.8599446421510478
3373/5869
Accuracy 0.8598873406463089
3374/5869
Accuracy 0.8599091088717645
3375/5869
Accuracy 0.8599111111111111
3376/5869
Accuracy 0.85989336492891
3377/5869
Accuracy 0.8599151120323759
3378/5869
Accuracy 0.859897375172686
3379/5869
Accuracy 0.8598599191082175
3380/5869
Accuracy 0.8598027613412229
3381/5869
Accuracy 0.8598442275460909
3382/5869
Accuracy 0.8598462448255471
3383/5869
Accuracy 0.8598679672874174
3384/5869
Accuracy 0.859869976359338
3385/5869
Accuracy 0.8598719842442146
3386/5869
Accuracy 0.8598543020279582
3387/5869
Accuracy 0.8598563133549847
3388/5869
Accuracy 0.8598386462022826
3389/5869
Accuracy 0.8598406609619357
3390/5869
Accuracy 0.8598623402163225
3391/5869
Accuracy 0.8598840066843606
3392/5869
Accuracy 0.8599056603773585
3393/5869
Accuracy 0.8598683564200805
3394/5869
Accuracy 0.8598114319387153
3395/5869
Accuracy 0.8597545

In [ ]:
from sklearn.metrics import f1_score,matthews_corrcoef
pred_df = pd.read_csv(test_pred_csv_file, header=None)
if (len(pred_df) != len(df_test)):
  print("MISMATCH")
print("f1_score",f1_score(pred_df[1].tolist(), pred_df[0].tolist()))
print("MCC", matthews_corrcoef(pred_df[1].tolist(), pred_df[0].tolist()))
# f1_score(pred_df[1].tolist(), pred_df[0].tolist(), average=None)

f1_score 0.8683645882504398
MCC 0.7272838572815044


## Test - Prediction 01 - 01 = 360k model, no freeze on params
bert-base-uncased-2epoch-LR5e-5

In [ ]:
df_test = pd.read_csv(data_folder + "/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/test_all_data_corrected_metadata.csv" , usecols = ['tweet', 'label'])
test_pred_csv_file="/content/drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_Similarity_With_Metadata_360k_Tweets/Classifier/01-nofreeze-twiBot-360k-similarity-metadata-2epoch_LR5e-5_bert_base_uncased/classifier_model/checkpoint-22000/pred_test_twiBot-360k-similarity-metadata-2epoch_LR5e-5_bert_base_uncased.csv"

In [ ]:
print(len(df_test))
print(df_test.iloc[0])
df_test = df_test.sample(frac=1, random_state=1).reset_index(drop=True)
print(df_test.iloc[0])

88035
tweet    SharonIsrael10 unprotected unverified defprof nondefimg 16596 16944 1 42684 49757 @ArtValley818_ CaliRed.
label                                                                                                            1
Name: 0, dtype: object
tweet    StarleeKine unprotected verified nondefprof nondefimg 50338 706 715 27692 18609 @GudGreaf @WeHaveSnacks I get the lack of enthusiasm. it is not an ideal situation. But it is the only way forward in this moment
label                                                                                                                                                                                                                    0
Name: 0, dtype: object


In [ ]:
#Check if we need to freeze params
model.eval()
pred_curated = []
batch_size = 15 # 88035 / 15 = 5869
count = 0
for i in range(0, len(df_test), batch_size):
  print("{}/{}".format(int((i/batch_size)+1), int(len(df_test)/batch_size)))
  pt_batch = tokenizer(
      list(df_test[i:i+batch_size].tweet), #["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
      padding=True,
      truncation=True,
      max_length=512,
      return_tensors="pt"
  )
  with torch.no_grad():
    prediction_logits = model(**pt_batch)
    prediction_results = torch.softmax(prediction_logits[0], dim=1)
    prediction_results = torch.argmax(prediction_results, dim=1)
    pred_curated.extend([[i,j] for i,j in zip(prediction_results.tolist(),df_test[i:i+batch_size].label.tolist())])
    # print(prediction_results.numpy().shape, np.array(df_test[i:i+batch_size].label).shape )
    # print("prediction_results", prediction_results)
    # print("actual_results", df_test[i:i+batch_size].label)
    count += np.sum(prediction_results.numpy() == np.array(df_test[i:i+batch_size].label))
    # print("count", count, "curr_total", (i+batch_size))
    print("Accuracy", count/(i+batch_size))
  # if (int((i/batch_size)+1)%5 ==0):
    
    # print("last batch saved", str(int((i/batch_size)+1)%100))
with open(test_pred_csv_file,"a") as f:
  # x=pickle.load(f)
  # x.extend(pred_cul)
  # pickle.dump(pred_cul, f)
  # pred_cul = []
  f_writer=csv.writer(f)
  f_writer.writerows(pred_curated)
  pred_curated =[]
print("Final Accuracy=", count/len(df_test))

Streaming output truncated to the last 5000 lines.
Accuracy 0.8470029673590505
3371/5869
Accuracy 0.8469890240284782
3372/5869
Accuracy 0.8469750889679716
3373/5869
Accuracy 0.8469413973712817
3374/5869
Accuracy 0.8469670025686623
3375/5869
Accuracy 0.8469728395061729
3376/5869
Accuracy 0.8469391785150079
3377/5869
Accuracy 0.8469845030105616
3378/5869
Accuracy 0.8469705940398657
3379/5869
Accuracy 0.8469566933017658
3380/5869
Accuracy 0.8469230769230769
3381/5869
Accuracy 0.8469683525584146
3382/5869
Accuracy 0.8469741770155726
3383/5869
Accuracy 0.8469799980293625
3384/5869
Accuracy 0.8469661150512214
3385/5869
Accuracy 0.8469719350073855
3386/5869
Accuracy 0.8469580626107501
3387/5869
Accuracy 0.8469441984056687
3388/5869
Accuracy 0.8469106650924833
3389/5869
Accuracy 0.8468771515688011
3390/5869
Accuracy 0.8469026548672567
3391/5869
Accuracy 0.8469281431239556
3392/5869
Accuracy 0.8469536163522012
3393/5869
Accuracy 0.846959426269771
3394/5869
Accuracy 0.8469063052445492
3395/5869


In [ ]:
pred_df = pd.read_csv(test_pred_csv_file, header=None)
if (len(pred_df) != len(df_test)):
  print("MISMATCH")
print("f1_score",f1_score(pred_df[1].tolist(), pred_df[0].tolist()))
print("MCC", matthews_corrcoef(pred_df[1].tolist(), pred_df[0].tolist()))
# f1_score(pred_df[1].tolist(), pred_df[0].tolist(), average=None)

f1_score 0.8507313829787234
MCC 0.6952919178055991
